# ライントレース+物体検知

## 物体検知側の下準備

In [1]:
from jetbot import ObjectDetector

# 物体検知モデルの読み込み
model = ObjectDetector('../note_wavashare/object_following/ssd_mobilenet_v2_coco.engine')

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

mean_obj = 255.0 * np.array([0.485, 0.456, 0.406])
stdev_obj = 255.0 * np.array([0.229, 0.224, 0.225])
normalize_obj = torchvision.transforms.Normalize(mean_obj, stdev_obj)

def preprocess_obj(camera.value):
    global device, normalize_obj
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [ ]:
from jetbot import bgr8_to_jpeg

blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):F
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection


def execute(change):
    image = change['new']
    
    # execute collision model to determine if blocked
   # collision_output = collision_model(preprocess(image)).detach().cpu()
   # prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
   # blocked_widget.value = prob_blocked
    
    # turn left if blocked
   # if prob_blocked > 0.5:
     #   robot.left(0.3)
     #   image_widget.value = bgr8_to_jpeg(image)
     #   return
        
    # compute all detected objects
    detections = model(image)
    
    # draw all detections on image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
    
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    
        
    # otherwise go forward if no target detected
    #if det is None:
        #robot.forward(float(speed_widget.value))
        
    # otherwise steer towards target
   # else:
        # move robot forward and steer proportional target's x-distance from center
        #center = detection_center(det)
       # robot.set_motors(
         #   float(speed_widget.value + turn_gain_widget.value * center[0]),
         #   float(speed_widget.value - turn_gain_widget.value * center[0])
       # )
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})
camera.observe(execute, names='value')

In [ ]:
import time

camera.unobserve_all()
time.sleep(1.0)
# robot.stop()
camera.stop()

## ライントレース側の下準備

In [1]:
# インポート
import torchvision
import torch

# 自分で作成したライントレースのモデルを読み込む
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
model.load_state_dict(torch.load('best_steering_models/[nvidia_circuit-ver1.2]_res18.pth'))

In [3]:
# GPU側に転送
device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

In [4]:
# カメラから読み込む映像への前処理を実装
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [5]:
# カメラからの映像をセルの下に出力
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera()
image_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [6]:
# 駆動系周りの準備
from jetbot import Robot

robot = Robot()  # 駆動系を制御できるモジュールのインポート

# 制御するバーの設定
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

# 描画
display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

# 自Jetbotにおいては、以下の設定だと良い感じに動く
# speed gain = 0.1
# steering gain = 0.05
# steering kd = 0
# steering gain = 0

In [ ]:
# 現在の動作を示すバーを描画
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

## 本処理

In [9]:
angle = 0.0
angle_last = 0.0

def execute(change):
   
    global angle, angle_last
    image = change['new']  # 映像読み込み
    #-----ライントレース部分----------#
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    #----------------------------------------------#
    
    #----------------物体検知部分-------------------#
    # カメラの映像内において、検知出来た全ての物体を、変数detectionに入れる
    # detections = model(image)
    
    
    
    
    
execute({'new': camera.value})

Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing.

We accomplish that with the observe function.

>WARNING: This code will move the robot!! Please make sure your robot has clearance and it is on Lego or Track you have collected data on. The road follower should work, but the neural network is only as good as the data it's trained on!

In [10]:
camera.observe(execute, names='value')

In [11]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

In [12]:
camera.stop()